In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report, accuracy_score, confusion_matrix

import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline

from unsloth import is_bfloat16_supported
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
system_prompt = (
    "You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment."
)

In [4]:
def prepare_prompt(row, train):
    prompt = system_prompt + "\n\n ### Input: " + row["Sentence"] + "\n\n ### Response: "
    if train:
         prompt = prompt + row["Sentiment"]
    return prompt

In [8]:
train_df = pd.read_csv("/content/train_data.csv")
val_df = pd.read_csv("/content/validation_data.csv")
test_df = pd.read_csv("/content/test_data.csv")

train_df["text"] = train_df.apply(lambda row: prepare_prompt(row, True), axis=1)
val_df["text"] = val_df.apply(lambda row: prepare_prompt(row, True), axis=1)
test_df["text"] = test_df.apply(lambda row: prepare_prompt(row, False), axis=1)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print(train_df["text"][101])
print(val_df["text"][101])
print(test_df["text"][101])

You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.

 ### Input: Ramirent 's net sales in the second quarterended June 30 were EURO 128.7 million about U.S. $ 163 million , a 3.3-percent increase compared with EURO 124.6 million for thesecond quarter last year .

 ### Response: positive
You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.

 ### Input: Vanhanen said the s

In [5]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')


    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [6]:
def predict(model, tokenizer):
    y_pred = []
    FastLanguageModel.for_inference(model)
    for i in tqdm(range(len(test_df))):
        prompt = test_df["text"][i]
        inputs = tokenizer(
            [
              prompt
            ], return_tensors = "pt").to("cuda")

        outputs = model.generate(**inputs, max_new_tokens = 1, use_cache = True)
        result = tokenizer.batch_decode(outputs)
        answer = result[0].split("### Response: ")[1].strip()
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

In [11]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
y_pred = predict(model, tokenizer)

100%|██████████| 877/877 [02:22<00:00,  6.17it/s]


In [10]:
y_true = test_df["Sentiment"]
evaluate(y_true, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.536

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       129
           1       0.54      1.00      0.70       470
           2       0.00      0.00      0.00       278

    accuracy                           0.54       877
   macro avg       0.18      0.33      0.23       877
weighted avg       0.29      0.54      0.37       877


Confusion Matrix:
[[  0 129   0]
 [  0 470   0]
 [  0 278   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params = {trainable_params} | All params = {all_param} | Trainable % = {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

Trainable params = 41943040 | All params = 3800305664 | Trainable % = 1.10


In [14]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        seed = 3407,
        output_dir = "./output",
        logging_dir="./logs"
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/4089 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/876 [00:00<?, ? examples/s]

In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,089 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,022
 "-____-"     Number of trainable parameters = 41,943,040


Epoch,Training Loss,Validation Loss
0,0.688800,0.645814
1,0.464900,0.638586


In [16]:
output_dir = "./trained_model"
trainer.save_model()
tokenizer.save_pretrained(output_dir)

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/tokenizer.model',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [17]:
print(test_df["text"][10])
print(test_df["Sentiment"][10])

You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.

 ### Input: The board of directors also proposed that a dividend of EUR0 .20 per outstanding share be paid .

 ### Response: 
neutral


In [18]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    test_df["text"][10]
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1, use_cache = True)
result = tokenizer.batch_decode(outputs)
result

['<s> You are an expert in financial sentiment analysis. Financial sentiment is defined as the tone and outlook expressed in financial texts. Sentiments can have a significant impact on market perceptions and investor decisions. Classify the following input text as positive, neutral, or negative based on its potential impact on financial markets and investor sentiment.\n\n ### Input: The board of directors also proposed that a dividend of EUR0 .20 per outstanding share be paid .\n\n ### Response:  neutral']

In [19]:
y_pred = predict(model, tokenizer)


100%|██████████| 877/877 [02:23<00:00,  6.11it/s]


In [20]:
evaluate(y_true, y_pred)

Accuracy: 0.843

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.57      0.63       129
           1       0.85      0.87      0.86       470
           2       0.87      0.92      0.90       278

    accuracy                           0.84       877
   macro avg       0.81      0.79      0.80       877
weighted avg       0.84      0.84      0.84       877


Confusion Matrix:
[[ 74  49   6]
 [ 30 409  31]
 [  1  21 256]]


Accuracy improved from 53.6% to 84.3%

In [21]:
evaluation = pd.DataFrame({'text': test_df["text"],
                           'y_true':y_true,
                           'y_pred': y_pred},
                         )
evaluation.to_csv("mistral_predictions.csv", index=False)